In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from os.path import isfile
import pickle
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import re
import numpy as np

%matplotlib inline

sns.set_style(style='white')
sns.set_palette(sns.color_palette("Dark2"))

# **Анализ данных.**

In [2]:
def proceed_cat_str(in_cat: pd.Series):
    
    res_cat = in_cat.map(
        lambda x: str.upper(x).strip() if type(x) is str else np.NaN
    )

    return res_cat

In [3]:
src_train = pd.read_csv('./data/train.csv')
src_test = pd.read_csv('./data/test.csv')

In [4]:
src_df = pd.concat([src_train, src_test], ignore_index=True).reset_index()
src_df.drop(['index'], inplace=True, axis=1)
print(src_df.shape)

(1309, 12)


In [5]:
print(src_df.isna().sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64


Разберемся с названиями столбцов и что это значит.
1. Survived - выжил человек или нет.
2. Pclass - класс каюты.
3. Name - имя.
4. Sex - пол.
5. Age - возраст.
6. SibSp - количество родственников второго порядка. Муж, жена, братья, сестры.
7. Parch - количество родственников первоко порядка. Мать, отец, ребенок.
8. Ticket - номер билета.
9. Fare - цена билета.
10. Cabin - каюта.
11. Embarked - порт посадки. C - Cherbourg. Q - Queenstown. S - Southampton.

Пол кодируется очевидно.

In [6]:
sex_cat = {'male': 0, 'female': 1}
src_df['Sex'] = src_df['Sex'].map(sex_cat).astype('int')

Поскольку одно отсутствуещее значение цены билета принадлежит дядьке без родственников в третьем классе, присвоим ему среднее по третьему классу.

In [7]:
src_df['Fare'] = src_df['Fare'].fillna(src_df.groupby(['Pclass']).Fare.median()[3])

Пропущенное значение посадки можно просто нагуглить. Тупо в каком порту эти люди сели.

In [8]:
src_df['Embarked'] = src_df['Embarked'].fillna('S')

In [9]:
embarked_cat = dict(enumerate(src_df['Embarked'].astype('category').cat.categories))
src_df['Embarked'] = src_df['Embarked'].astype('category').cat.codes

Интересно, что есть нулевые цены билетов. Нужно взглянуть.

In [10]:
zero_fare_idx = src_df.loc[src_df['Fare'] == 0].index
print(src_df.loc[src_df['Fare'] == 0][['Pclass', 'Parch', 'SibSp']].sort_values(by=['Pclass']))

      Pclass  Parch  SibSp
1263       1      0      0
263        1      0      0
822        1      0      0
815        1      0      0
1157       1      0      0
633        1      0      0
806        1      0      0
277        2      0      0
413        2      0      0
466        2      0      0
481        2      0      0
674        2      0      0
732        2      0      0
179        3      0      0
271        3      0      0
302        3      0      0
597        3      0      0


Похоже, что это кто-то из гостей или команды, или что-то вроде. Поставлю среднее.

In [11]:
med_fare_1 = src_df.loc[src_df['Fare'] != 0].groupby(['Pclass']).Fare.median()[1]
med_fare_2 = src_df.loc[src_df['Fare'] != 0].groupby(['Pclass']).Fare.median()[2]
med_fare_3 = src_df.loc[src_df['Fare'] != 0].groupby(['Pclass']).Fare.median()[3]

fare_medians = {1: med_fare_1, 2: med_fare_2, 3: med_fare_3}

In [12]:
for index in zero_fare_idx:
    src_df.loc[index, 'Fare'] = fare_medians[src_df.loc[index, 'Pclass']]

Имена. Тут интересно. <br>
    <br>Mr. Обращение к мужчине любого возраста и семейного положения.
    <br>Master. Обращение к мальчику до 16 лет.
    <br>Mrs. Обращение к женщине, о которой нам точно известно что она замужем или вдова.
    <br>Miss. Mlle. Обращение к женщине, о которой нам точно известно что она не замужем.
    <br>Ms. Обращение к женщине, семейный статус которой нам не известен.
    <br>Rev. Обращение к священослужителю.
    <br>Dr. Человек с докторской степенью.
    <br>Sir. Lady. Don. Dona. Jonkheer. Countess. Mme. Дворянские титулы или люди "высокого" происхождения.
    <br>Capt. Col. Major. Офицерские звания.
    <br><br>Это может пригодиться для импутации возраста. Дополнительный фактор определения влияния цвета штанов на выживаемость.
    

In [13]:
def proceed_name(name: str):
    if 'MR.' in name:
        return 0
    elif 'MRS.' in name:
        return 1
    elif ('MISS.' in name) or ('MLLE.' in name):
        return 2
    elif 'MASTER.' in name:
        return 3
    elif 'REV.' in name:
        return 4
    elif 'MS.' in name:
        return 5
    elif 'DR.' in name:
        return 6
    elif ('SIR.' in name) or ('LADY.' in name) or ('DON.' in name) or ('DONA.' in name) or ('JONKHEER.' in name) or ('COUNTESS.' in name) or ('MME.' in name):
        return 7
    elif ('COL.' in name) or ('MAJOR.' in name) or ('CAPT.' in name):
        return 8
    else:
        return 9

In [14]:
src_df['Name'] = proceed_cat_str(src_df['Name']).map(proceed_name).astype('int')

Из билета, вероятно, тоже можно что-то извлечь. Но я не нашел однозначного алгоритма формирования номера. Поэтому выкину.

Каюта. Не бесполезный параметр. Но класс у нас уже есть, а как ТОЧНО определить каюту я не нашел. Можно пойти длинным путем и попробовать намайнить в интернете или поискать некую взаимосвязь между имеющимися данными и каютой. Но в данной конкретной работе я смысла в этом не вижу. Поэтому удалю.

In [15]:
src_df.drop(['Ticket', 'Cabin'], inplace=True, axis=1)

In [16]:
src_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,0,0,22.0,1,0,7.2500,2
1,2,1.0,1,1,1,38.0,1,0,71.2833,0
2,3,1.0,3,2,1,26.0,0,0,7.9250,2
3,4,1.0,1,1,1,35.0,1,0,53.1000,2
4,5,0.0,3,0,0,35.0,0,0,8.0500,2


In [17]:
print(src_df.isna().sum())

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Fare             0
Embarked         0
dtype: int64


Осталось импутировать возраст. Буду импутировать средним. 

Посмотрим как выглядит средний возраст по имени и классу. Сюда бы, конечно, еще бы родственников. Но датасет слишком маленький.

In [18]:
mean_age_by_name_and_class = src_df.loc[~src_df['Age'].isna()].groupby(['Name', 'Pclass']).max()['Age']
print(mean_age_by_name_and_class)

Name  Pclass
0     1         80.0
      2         70.0
      3         74.0
1     1         76.0
      2         60.0
      3         63.0
2     1         63.0
      2         50.0
      3         45.0
3     1         13.0
      2          8.0
      3         14.5
4     2         57.0
5     2         28.0
6     1         53.0
      2         54.0
7     1         49.0
8     1         70.0
Name: Age, dtype: float64


Немного неудобно смотреть, но всё логично. Аристократы и офицеры в первом, ученые в первом-втором, женщины с неизветным статусом и священники во втором, ну и все остальные кому как повезло.

In [19]:
age_na_idx = src_df.loc[src_df['Age'].isna()].index
print(age_na_idx)

Int64Index([   5,   17,   19,   26,   28,   29,   31,   32,   36,   42,
            ...
            1256, 1257, 1271, 1273, 1275, 1299, 1301, 1304, 1307, 1308],
           dtype='int64', length=263)


In [20]:
for index in age_na_idx:
    name_v = int(src_df.loc[index, 'Name'])
    class_v = int(src_df.loc[index, 'Pclass'])
    if name_v in mean_age_by_name_and_class:
        if class_v in mean_age_by_name_and_class[name_v]:
            age_m = np.int(mean_age_by_name_and_class[name_v][class_v])
            src_df.loc[index, 'Age'] = age_m
        else:
            print("{}. There is no key {} in {}".format(index, name_v, mean_age_by_name_and_class[name_v]))
            print(src_df.loc[index, ])
            break
    else:
        print("{}. There is no key {} in mean_age_by_name_and_class.".format(index, name_v))
        break


979. There is no key 5 in Pclass
2    28.0
Name: Age, dtype: float64
PassengerId    980.00
Survived          NaN
Pclass           3.00
Name             5.00
Sex              1.00
Age               NaN
SibSp            0.00
Parch            0.00
Fare             7.75
Embarked         1.00
Name: 979, dtype: float64


In [21]:
print(src_df['Pclass'].max())

3
